# Create Video using Nova Reel
---

## 0. 환경 설정

In [1]:
# ! pip install Pillow
# # ! pip install boto3==1.36.14
# ! pip install boto3 -U

In [2]:
%load_ext autoreload
%autoreload 2

from local_util import (
        resize_with_padding, call_video_creation, show_status_video_creation,
        show_date_str, download_file_from_s3
) 


## 1. Resize Image

In [3]:

# 사용 예시
input_image = "input_img/hasun_baby.jpg"  # 원본 이미지 경로
output_image = "output_img/hasun_baby_resize.jpg"  # 저장할 이미지 경로

# 함수 실행
resize_with_padding(input_image, output_image)

## 2. Call Bedrock API

### Setup Parameters

In [4]:
from textwrap import dedent

# bucket_name = "s3://bedrock-video-generation-us-east-1-uaam38"
bucket_name = "bedrock-video-generation-us-east-1-uaam38"
text_prompt = dedent("""
    A consistent professional-looking person in business attire celebrating their 3rd work anniversary in an office setting, 
    maintaining stable facial features, 
    A happy employee smiling and holding a small cake with '3 years' written, 
    confetti falling slowly, colleagues congratulating in the background, warm office lighting, 
    cinematic quality, steady camera movement, high coherence between frames, consistent character appearance, 
    6 second video clip, smooth and slow motion, high temporal consistency
""")

print(text_prompt)


A consistent professional-looking person in business attire celebrating their 3rd work anniversary in an office setting, 
maintaining stable facial features, 
A happy employee smiling and holding a small cake with '3 years' written, 
confetti falling slowly, colleagues congratulating in the background, warm office lighting, 
cinematic quality, steady camera movement, high coherence between frames, consistent character appearance, 
6 second video clip, smooth and slow motion, high temporal consistency



### Call Bedrock API

In [5]:
! pip list | grep boto3

boto3                                   1.38.35


In [6]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
import boto3


# Create the Bedrock Runtime client.
bedrock_runtime = boto3.client("bedrock-runtime")

response = call_video_creation(
                    bedrock_runtime = bedrock_runtime, 
                    bucket_name = bucket_name, 
                    input_image = output_image, 
                    text_prompt = text_prompt
                    )



ValidationException: An error occurred (ValidationException) when calling the StartAsyncInvoke operation: The provided model identifier is invalid.

In [19]:
response

{'ResponseMetadata': {'RequestId': 'ab4d1524-dc72-40a8-a9a5-25c5b2c14381',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 07 Feb 2025 14:18:07 GMT',
   'content-type': 'application/json',
   'content-length': '84',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ab4d1524-dc72-40a8-a9a5-25c5b2c14381'},
  'RetryAttempts': 0},
 'invocationArn': 'arn:aws:bedrock:us-east-1:057716757052:async-invoke/0ktleg368agy'}

In [20]:
invocation_arn = response["invocationArn"]
print(invocation_arn)


arn:aws:bedrock:us-east-1:057716757052:async-invoke/0ktleg368agy


## 3. Show job status

In [21]:
import time, os

status = "InProgress"
while ( status != "Completed"):
    status, invocation = show_status_video_creation(
                                bedrock_runtime = bedrock_runtime, 
                                invocationArn = invocation_arn
                                )
    print("status: ", status)
    time.sleep(5)

output_videoo_s3 = invocation["outputDataConfig"]["s3OutputDataConfig"]["s3Uri"]
output_videoo_s3 = os.path.join(output_videoo_s3, "output.mp4")
print("output_videoo_s3: ", output_videoo_s3)


{
  "ResponseMetadata": {
    "RequestId": "d6a347e4-d61c-4f66-9952-440b7a1d38e0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 07 Feb 2025 14:18:10 GMT",
      "content-type": "application/json",
      "content-length": "437",
      "connection": "keep-alive",
      "x-amzn-requestid": "d6a347e4-d61c-4f66-9952-440b7a1d38e0"
    },
    "RetryAttempts": 0
  },
  "invocationArn": "arn:aws:bedrock:us-east-1:057716757052:async-invoke/0ktleg368agy",
  "modelArn": "arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-reel-v1:0",
  "clientRequestToken": "afa3cda9-9254-41ed-b29c-5743415df848",
  "status": "InProgress",
  "submitTime": "2025-02-07 14:18:06+00:00",
  "lastModifiedTime": "2025-02-07 14:18:06+00:00",
  "outputDataConfig": {
    "s3OutputDataConfig": {
      "s3Uri": "s3://bedrock-video-generation-us-east-1-uaam38/0ktleg368agy"
    }
  }
}
status:  InProgress
{
  "ResponseMetadata": {
    "RequestId": "b3133d57-2904-42b4-94c3-25b57afdcc01",
    "HTTPStatus

## 4. Downlaod video and Play video

In [24]:
split_str = output_videoo_s3.split('/')
object_key = os.path.join(split_str[-2], split_str[-1])
print("object_key: ", object_key)

local_output_folder = "output_video"
os.makedirs(local_output_folder, exist_ok=True)

formatted_datetime = show_date_str()
output_file_name = os.path.join(local_output_folder, f"{formatted_datetime}.mp4")
print("output_file_name: ", output_file_name)



object_key:  0ktleg368agy/output.mp4
2025-02-07-14-23
output_file_name:  output_video/2025-02-07-14-23.mp4


In [25]:
download_file_from_s3(bucket_name=bucket_name, object_key=object_key, local_file_path=output_file_name)


Video downloaded successfully to output_video/2025-02-07-14-23.mp4
